In [ ]:
"""
look at potential bug in profile making
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import itertools

import distributed
import glob

from tqdm.notebook import tqdm


In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
    ens_run = 'r1i1p1f2'
    to2300 = False
elif mod in ['GISS-E2-1-H']:
    ens_run = 'r1i1p1f2'
    to2300 = True
elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
    ens_run = 'r1i1p1f1'
    to2300 = True
elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
    ens_run = 'r1i1p1f1'
    to2300 = False
elif mod == 'UKESM1-0-LL':
    ens_run = 'r4i1p1f2'
    to2300 = True     
elif mod == 'CESM2':
    ens_run = 'r11i1p1f1'
    to2300 = False        


if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_isf+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
inputpath_data_orig='/bettik/jourdai1/OCEAN_DATA_CMIP6_STEREO/'+mod+'/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
scenario = 'historical'
T_list = []
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
    
    T_file = xr.open_mfdataset(inputpath_profiles + mod + '/T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_*.nc', combine='nested', concat_dim='time')
    T_list.append(T_file.mean('time').assign_coords({'model': mod}))

T_all = xr.concat(T_list, dim='model').load()

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.1

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(T_all.Nisf):

    ax[i] = f.add_subplot(8,8,i+1)
    for m,mod in enumerate(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']):
        
        for tt in range(1980,2015):
            
            if mod == 'CNRM-CM6-1':
                ccolor='k'
            else:
                ccolor='grey'
                
            T_data = T_all['thetao'].sel(Nisf = kisf, model=mod)
            ax[i].plot(T_data,T_data.z, color=ccolor, alpha=alpha_val)



            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'T_profiles_CMIP_'+mod+'_'+scenario+'_all.png', dpi=300)

In [ ]:
mod = 'CNRM-CM6-1'
scenario = 'historical'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

check_2D_field = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_1980.nc')

In [ ]:
check_2D_field['theta_in'].where(file_isf['ISF_mask'] == 66, drop=True).plot()

In [ ]:
kisf= 66
amask_sum.sel(Nisf=kisf).plot()
mask_sum2.sel(Nisf=kisf).plot()